# LU Decomposition

### Matrix Rank

Let $\mathbb{F}$ be a field and $m, n \in \mathbb{N}$. The __matrix rank__ of $A \in \mathbb{F}^{m \times n}$, denoted $\mathrm{rank} A$, is the least number $r \in \mathbb{N}_0$ such that there exist $u_1, \dots, u_r \in \mathbb{F}^m$ and $v_1, \dots, v_r \in \mathbb{F}^n$ satisfying

$$ A = \sum_{k=1}^r u_k v_k^\top $$

The set of matrices of rank not exceeding $r \in \mathbb{N}_0$ is denoted as

$$ \mathbb{F}^{m \times n}_r = \{ A \in \mathbb{F}^{m \times n} \ | \ \mathrm{rank} A \le r \} = \bigg\{ \sum_{k=1}^r u_k v_k^\top \ \bigg| \ u_1, \dots, u_r \in \mathbb{F}^m,\ v_1,\dots, v_r \in \mathbb{F}^n \bigg\} $$

The only matrix with rank zero is the zero matrix, so $\mathbb{F}^{m \times n}_0 = \{ 0 \}$.

Some properties of the matrix rank:

- $\mathrm{rank} A = \mathrm{rank} A^\top$
- $\mathrm{rank} A \le \min\{m, n\}$ for any $A \in \mathbb{F}^{m \times n}$
- the rank of any submatrix of $A$ does not exceed the rank of $A$
- $A \in \mathbb{F}^{m \times n}_p$ and $B \in \mathbb{F}^{m \times n}_q$ with $p, q \in \mathbb{N}_0$ $\implies$ $A + B \in \mathbb{F}^{m \times n}_{p+q}$
- $A \in \mathbb{F}^{m \times k}_p$ and $B \in \mathbb{F}^{k \times n}_q$ with $p, q \in \mathbb{N}_0$ $\implies$ $AB \in \mathbb{F}^{m \times n}_r$ with $r = \min\{p,q\}$
- $A \in \mathbb{F}^{m \times n}$ then $\mathrm{rank} A = \mathrm{rank} PAQ$ for any invertible matrices $P \in \mathbb{F}^{m \times m}$ and $Q \in \mathbb{F}^{n \times n}$

A representation of $A \in \mathbb{F}^{m \times n}$ for $m, n \in \mathbb{N}$ of the form $A = UV^\top$ with $U \in \mathbb{F}^{m \times r}$ and $V \in \mathbb{F}^{n \times r}$ for some $r \in \mathbb{N}$ is called __rank-r factorization__ of $A$.

$A \in \mathbb{F}^{m \times n}_r \iff$ there exists a rank-$r$ factorization of $A$.

The rank-$r$ factorization is not unique though since $UV^\top = \tilde{U} \tilde{V}^\top$ with $\tilde{U} = UT$ and $\tilde{V} = V(T^{-1})^\top$ for any invertible matrix $T \in \mathbb{F}^{r \times r}$.

### LU Decomposition

__Lemma 1__ (inversion of a single step of Gaussian elimination) \
For $m \in \mathbb{N}$ and $k \in \{1, \dots, m\}$, let $I \in \mathbb{F}^{m \times m}$ denote the identity matrix, $e_k \in \mathbb{F}^m$ be column $k$ of $I$ and $\ell \in \mathbb{F}^m$ with zeroes as entries 1 to $k$. Then the matrix $I - \ell e_k^\top$ is invertible with $\big(I - \ell e_k^\top \big)^{-1} = I + \ell e_k^\top$.

__Lemma 2__ (representation of $r$ steps of Gaussian elimination) \
For $m \in \mathbb{N}$ and $r \in \{1, \dots, m\}$ and $\ell_1, \dots, \ell_r \in \mathbb{F}^m$ such that components 1 to $k$ of $\ell_k$ are all zero for each $k \in \{1, \dots, r\}$. Then $(I + \ell_1 e_1^\top) \cdots (I + \ell_r e_r^\top) = I + \sum_{k=1}^r \ell_k e_k^\top$.

$A = LU \in \mathbb{F}^{m \times n}$ is called an __r-step LU decomposition__ with $L$ and $U$ as __r-step LU factors__ if
- for $r < \min\{m, n\}$: $L \in \mathbb{F}^{m \times m}$ is of the form $L = \begin{pmatrix} L_1 & \\ L_2 & I_2 \end{pmatrix}$ and $U \in \mathbb{F}^{m \times n}$ is of the form $\begin{pmatrix} U_1 & U_2 \\ & S \end{pmatrix}$, where $L_1 \in \mathbb{F}^{r \times r}$ is a unit lower-triangular matrix, $U_1 \in \mathbb{F}^{r \times r}$ an upper-triangular matrix with no zeros on the diagonal, $I_2$ the identity matrix of order $m-r$ and $L_2 \in \mathbb{F}^{(m-r) \times r}$, $U_2 \in \mathbb{F}^{r \times (m-r)}$ and $S \in \mathbb{F}^{(m-r) \times (n-r)}$ are arbitrary.
- for $r = \min\{m, n\}$: $L$ is a unit lower-triangular matrix and $U$ is an upper-triangular matrix with no zeros on the diagonal.

When $r < \min\{m, n\}$ and $S$ is nonzero, $S$ is called __r-step Schur complement__ and the r-step LU decomposition is called __incomplete__.
When $r = \min\{m, n\}$ and $S$ is zero then the r-step LU decomposition is called __complete__.

For any $m, n \in \mathbb{N}$, $p \in \{ 1, \dots, m\}$, $q \in \{1, \dots, n\}$ and $A \in \mathbb{F}^{m \times n}$, the submatrix of $A$ formed by rows $1, \dots, p$ and columns $1, \dots, q$ is called the __leading principal submatrix__ of size $p \times q$ of $A$.

In [1]:
function rStepLU(A::Matrix{T}, r::Union{Integer, Nothing}=nothing) where {T}
    m, n = size(A)
    min_dim = min(m, n)

    # promotes the element type of input matrix
    UT = typeof(oneunit(T) - oneunit(T) * oneunit(T) / oneunit(T))
    LT = typeof(oneunit(UT) / oneunit(UT))
    T = promote_type(T, LT, UT)

    # validate rank
    if r == nothing
        r = min_dim
    elseif r < 1 || r > min_dim
        throw(ArgumentError("rank does not satisfy 1 ≤ r ≤ $min_dim: got $r"))
    end

    # initialization of U and L
    U = convert(Matrix{T}, copy(A))
    L = zeros(T, (m, m))
    for i = 1:m
        L[i,i] = oneunit(T)
    end

    for k = 1:r
        # step k of Gaussian elimination
        for i = (k+1):m
            if U[k,k] == zero(T)
                return L, U
            end

            # compute elimination coefficient for row i
            L[i,k] = U[i,k] / U[k,k]

            # eliminate component i in the current column of U
            U[i,k] = zero(T)

            # update row i of U to compute the corresponding row of the Schur complement
            for j = (k+1):n
                U[i,j] = U[i,j] - L[i,k] * U[k,j]
            end
        end
    end
    return L, U
end


rStepLUDecomposition (generic function with 2 methods)

In [15]:
m = 7; n = 5
A = rand(Float64, (m,n))

println("r-step approximation error")
println("r | MAE")
for r = 1:min(m,n)
    L, U = rStepLU(A, r)
    println(r, " | ", sum(abs.(A - L[:,1:r] * U[1:r,:])) / m / n)
end

r-step approximation error
r | MAE
1 | 0.1995592386779962
2 | 0.14329210369426387
3 | 0.15589645438787275
4 | 0.0839072980864766
5 | 1.9825411154020653e-17


## Pivoted LU Decomposition

## Implications of the pivoted LU decomposition for matrix rank